In [2]:
import os
import sys
import json
import pickle

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pandas as pd

import functools

import numpy as np
import math
import random
import scipy

import plotting
import utils
import yaml

2025-02-05 14:29:10.814093: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-05 14:29:10.827424: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738762150.843829   10923 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738762150.848861   10923 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-05 14:29:10.865813: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
CONFIG_FILE = 'configs/esann25/experiment_config.json'

with open(CONFIG_FILE, 'r') as f:
    config = json.load(f)

openai_models = config["openai_models"]
huggingface_models = config["huggingface_models"]
models = huggingface_models + openai_models
emb_dir = '../%s' % config['embedding_dir']

In [31]:
def get_dataset_stats(g_train, g_test, groups):
    stats = {'train': {group: 0 for group in groups}, 'test': {group: 0 for group in groups}}
    df = pd.DataFrame(stats)
    
    if g_test.ndim == 1:
        # single-label
        if len(g_train) > 0:
            df.loc[groups[1], 'train'] = np.sum(g_train)
            df.loc[groups[0], 'train'] = len(g_train)-np.sum(g_train)
        df.loc[groups[1], 'test'] = np.sum(g_test)
        df.loc[groups[0], 'test'] = len(g_test)-np.sum(g_test)
        
        df.loc['total', :] = df.loc[groups, :].sum(axis=0)
    else:
        # multi-label
        for i, group in enumerate(groups):
            if len(g_train) > 0:
                df.loc[group, 'train'] = np.sum(g_train[:,i])
            df.loc[group, 'test'] = np.sum(g_test[:,i])
        df.loc['total', 'train'] = len(g_train)
        df.loc['total', 'test'] = len(g_test)
    return df

In [35]:
local_dir = '../../../data/jigsaw_bias'
X_train, emb_train, y_train, g_train, X_test, emb_test, y_test, g_test, groups, _, _  = utils.get_dataset_and_embeddings(emb_dir, 'jigsaw', huggingface_models[0], 'mean', 1, local_dir, sel_groups=None)

df = get_dataset_stats(g_train, g_test, groups)
df

{'target': np.float64(18505.0)}
[18.29310997]
{'target': np.float64(971.0)}
[18.43975283]


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-generator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using Bert with CUDA/GPU
no optmizer specified, default to AdamW
load precomputed embeddings for test set
load precomputed embeddings for train set


,train,test
female,40389.0,1953.0
male,30331.0,1547.0
transgender,1477.0,86.0
other_gender,1.0,0.0
white,15129.0,776.0
asian,2424.0,133.0
black,8498.0,421.0
latino,915.0,48.0
other_race_or_ethnicity,81.0,2.0
atheist,981.0,98.0


In [32]:
local_dir = '../../../data/bios_huggingface_merge.pkl'
X_train, emb_train, y_train, g_train, X_test, emb_test, y_test, g_test, groups, _, _  = utils.get_dataset_and_embeddings(emb_dir, 'bios-supervised', huggingface_models[0], 'mean', 1, local_dir, sel_groups=None)

df = get_dataset_stats(g_train, g_test, groups)
df

{'architect': np.float64(256.0), 'surgeon': np.float64(708.0), 'dentist': np.float64(117.0), 'teacher': np.float64(845.0), 'psychologist': np.float64(676.0), 'nurse': np.float64(432.0), 'photographer': np.float64(1195.0), 'physician': np.float64(824.0), 'attorney': np.float64(914.0), 'journalist': np.float64(1214.0)}
[26.41015625  8.91101695 58.97435897  7.30414201  9.38017751 15.24305556
  4.87196653  7.5157767   6.67724289  4.7800659 ]
{'architect': np.float64(118.0), 'surgeon': np.float64(359.0), 'dentist': np.float64(46.0), 'teacher': np.float64(454.0), 'psychologist': np.float64(313.0), 'nurse': np.float64(218.0), 'photographer': np.float64(599.0), 'physician': np.float64(379.0), 'attorney': np.float64(520.0), 'journalist': np.float64(632.0)}
[29.05084746  8.87743733 76.08695652  6.81057269 10.32907348 15.26605505
  4.91986644  8.35620053  5.81923077  4.61075949]


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-generator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using Bert with CUDA/GPU
no optmizer specified, default to AdamW
load precomputed embeddings for test set
load precomputed embeddings for train set


,train,test
male,4024.0,2008.0
female,2993.0,1538.0
total,7017.0,3546.0


In [33]:
X_train, emb_train, y_train, g_train, X_test, emb_test, y_test, g_test, groups, _, _  = utils.get_dataset_and_embeddings(emb_dir, 'crows_pairs', huggingface_models[0], 'mean', 1, local_dir, sel_groups=None)
g_train = np.asarray(g_train)
g_test = np.asarray(g_test)
df = get_dataset_stats(g_train, g_test, groups)
df

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-generator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using Bert with CUDA/GPU
no optmizer specified, default to AdamW
no loss function specified, default to cross entropy
load precomputed embeddings for test set


,train,test
black,0.0,315.0
white,0.0,382.0
asian,0.0,33.0
poor,0.0,128.0
rich,0.0,83.0
male,0.0,1300.0
female,0.0,703.0
"physical disability, sickness",0.0,40.0
mental_disability_illness,0.0,33.0
"foreigner, immigrant, undocumented",0.0,16.0


In [34]:
X_train, emb_train, y_train, g_train, X_test, emb_test, y_test, g_test, groups, _, _  = utils.get_dataset_and_embeddings(emb_dir, 'twitterAAE', huggingface_models[0], 'mean', 1, local_dir, sel_groups=None)
g_train = np.asarray(g_train)
g_test = np.asarray(g_test)
df = get_dataset_stats(g_train, g_test, groups)
df

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-generator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Using Bert with CUDA/GPU
no optmizer specified, default to AdamW
no loss function specified, default to cross entropy
load precomputed embeddings for test set


,train,test
aa,0.0,50000.0
white,0.0,50000.0
total,0.0,100000.0
